# Modelling of air-quality in Belgium for forecasting purposes using Deep Neural Networks

#### Konstantinos Theodorakos  
*Katholieke Universiteit Leuven*  
*Student of the Master Artificial Intelligence: Engineering and Computer Science*  
*Faculty of Engineering Science*  
*Departments of Computer Science & Electrical Engineering, ESAT-STADIUS*  
*Kasteelpark Arenberg 10*  
*Leuven (Heverlee), Belgium*  
*Email: Konstantinos.Theodorakos@student.kuleuven.be*

# BACKGROUND

## Particulate Matter

Particulate matter (see: https://en.wikipedia.org/wiki/Particulates ) or atmospheric aeorosol particles, is solid or liquid matter suspended in the Earth's atmosphere. Inhaling particulate matter may cause adverse effects on human and animal health: asthma, lung cancer, respiratory diseases, cardiovascular disease, premature delivery, birth defects, low birth weight, and premature death.

![Alt Text](pics/Airborne-particulate-size-chart.svg.png)
*Figure: 
Diameter-size distribution of airborne particles in micrometres (µm) (source: https://en.wikipedia.org/wiki/File:Airborne-particulate-size-chart.svg ).*

# 1. METHODS
## Data handling & time-series split

Sensor data points that were flagged as invalid were omitted entirely. For the 1-station ozone prediction experiments, linear interpolation is filling the missing values. For all the other 4+ station experiments, missing data points are covered by: (1) data from the geographically closest stations and (2) linear interpolation (in case there are still missing values).

For the model selection, a unique form of k-fold cross-validation was used: **time-series split** [3] see: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html ). In each split, shuffling is omitted, in order to retain the sequence of the full time-series. In addition, the model is trained with data of increasing total time-step count. The mean validation Mean Squared Error (MSE) determines whether a model is optimal and generalizes well. The "test" data is held-out from the start, never used in model selection. For all the experiments, time-series split had had 10-folds.
![title](pics/timeSeriesValidateSplit.png)


## Neuroevolution with "Island Transpeciation"

Neuroevolution [21] is an algorithmic method that utilizes *Evolutionary Algorithms* *(EA)* [18] to generate Artificial Neural Network (ANN) [22] parameters, topologies [5, 6, 10, 11, 12] and rules (see: https://en.wikipedia.org/wiki/Neuroevolution ). In order to find an optimal deep learning model for the time-series forecasting [8, 9] task, architecture search is performed in conjunction with hyper-parameter tuning [23, 34]. Speciation is the evolutionary process by which populations evolve to become distinct species (see: https://en.wikipedia.org/wiki/Speciation ). With *island speciation*, neuroevolution can be parallelized: Each island [24, 33] can be a distinct thread, process or even a different computing node. Even though the evolution progresses distinctly in each island, populations of meta-learning agents migrate periodically between islands.

Every island can have the same global optimization technique (with similar or different settings versus the other islands), a **species**. A global optimizer like Differential Evolution (DE) [17] (see: https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.optimize.differential_evolution.html ), maintains a number of different agents internally, that are used in a parameter search. An agent has a different internal representation (*genotype*) depending on the meta-heuristic algorithm. The actual bounded parameter set, the *phenotype*, is a candidate Deep Neural Network (DNN) [6] model architecture that we try to optimize. Within the DE implementation, there exist 2-way operations that can perform genotype-to-phenotype representation conversions and vice-versa.

Three types of global optimization EA islands were used:
- **Random (Rand)** [19] island. Creates a random model, by sampling the bounded hyper-parameter values from a uniform pseudo-random number generator. In this case genotype = phenotype and the island population is 1.

- **Differential Evolution** island. Genotype: energy (real values in [0,1]). Phenotype: the genotype values, scaled from [0, 1] into the bounds of each parameter.
![Alt Text](pics/Ackley.gif)
*Figure: Example of Differential Evolution optimizing the 2D Ackley function using a population of size 20 (source: https://en.wikipedia.org/wiki/Differential_evolution#/media/File:Ackley.gif ).*


- **Particle Swarm Optimization (PSO)** [16] island (see: https://pythonhosted.org/pyswarm/ ). Genotype: position (multi-dimensional representation of the parameters as real values) and velocity (rate of change of the positions).
![Alt Text](pics/ParticleSwarmArrowsAnimation.gif)
*Figure: A particle swarm searching for the global minimum of a function (source: https://en.wikipedia.org/wiki/Particle_swarm_optimization#/media/File:ParticleSwarmArrowsAnimation.gif ).*


In some cases, it is possible to convert an agent from one global optimization method to another. For example, to convert a DE to a PSO agent: the agent of the DE "species" is converted from the DE genotype to the common phenotype (by unscaling real values) and then is converted to the PSO genotype (real values = position & velocity). We define this conversion process as **island transpeciation** (see: http://www.websters1913.com/words/Transpeciate ). Loosely termed, transpeciation acts as a (lossy in some cases) *transformation function* between the agent genotypes of different global optimization functions. Transpeciation in conjunction with the island model, allow combination (via *migration*) and diversification (via unique evolution per island) of different model architecture search methods.

Worth noting is that in order to keep the populations stable on all islands, the migrating agents replace a random agent at the receiving island. In practice, it was a matter of replacing an agent entry on the internal data structures of the EA algorithms. For the local experiments, the island population was fixed to 5 agents.

In the future, there will be attempts to integrate additional meta-learning islands such as:
- Genetic Algorithm (GA) [13, 14, 15].
- (Multi-agent) Reinforcement Learning (RL) [4].
- Basin Hopping (BH) [20].
- Estimation of distribution algorithms (EDA) [26], or probabilistic model-building genetic algorithms (PMBGAs) [25].
- Tree-structured Parzen Estimator (TPE) [34].

![title](pics/agentIslandTranspeciation.png)

## Architecture search genes

The genes are expressed as bounded real or integer values. They can represent:
- Cells/layers of a sequential DNN.
- Various model hyper-parameters/rates.
- Training optimizers.

### Cell Sequence search

The candidate models contain 4 base layers:
- 3x Long-Short Term Memory (**LSTM**) [27].
- 1x **Fully Connected** cell at the end.

The **auxiliary/utility cells** are placed between the base layers. Auxiliary cells placements are determined purely by the evolutionary search process:

- Batch Normalization [36].
- Gaussian Noise [37].

![title](pics/cells.png)

### Hyper-Parameter search

The following **bounded parameters** (ranges expressed in the parentheses) are searched:
- Standard Deviation of the Gaussian noise.
- Dropout/Recurrent-dropout [35] rate: Fraction of neurons to randomly ignore while training; reduces overfitting.
- Cell/layer size.
- Max training epochs.
- Batch size per epoch.




![title](pics/hyperParameters.png)

### Optimizer search

**Optimizers** are the algorithms that guide the weight training process of an ANN. The following optimizer options are possible (https://keras.io/optimizers/):

- Adagrad (Adaptive Subgradient) [38].
- Adadelta [39].
- Adam, adamax [40].
- Adam (amsgrad version) [41].
- Nadam (adam with Nesterov momentum) [42].

![title](pics/optimizers.png)

It is a subset of the total optimizers available, the only valid options for Recurrent Neural Networks (RNN).

### Overfitting avoidance

**Early Stopping** and **Learning-Rate reduction on plateau** (see: https://keras.io/callbacks/ ) use the *loss* and *validation_loss* values of Mean Squared Error (MSE) during training, to approach a finely-trained model and to avoid overfitting.

### Parallelism
An asynchronous version of the **Master-Slave pattern** (see: http://charm.cs.uiuc.edu/research/masterSlave ) allowed for CPU and GPU parallelism of different island species. Agent migration is carried out by slaves (islands) sending candidates to the master (main CPU process). The master decides when and to which island to migrate an agent. **Message Passing Interface (MPI)** (see: https://www.mpi-forum.org/docs/ ) handles the communication between the islands and the master (see: https://mpi4py.readthedocs.io/en/stable/tutorial.html#point-to-point-communication ).

The parallel implementation was tested on a local workstation (6 physical CPU cores, 2x Cuda capable GPUs), as well as on the Amazon Elastic Compute Cloud (EC2), on a p3.8xlarge Amazon Machine Image (AMI) (see: https://aws.amazon.com/ec2/instance-types/p3/ ) containing 4x Tesla V100 GPUs. *Note: unfortunately, for now, the models/weights trained on the cloud are incompatible with the local tensorflow installation. After the completion of the "Python for HPC" seminar, the GPU cluster of the VSC super-computer will be utilized (see: https://www.vscentrum.be/ )*.

# 2. MODELS & RESULTS

# Particulate Matter 10 micrometer (μm) particle diameter (PM10) MIMO NARX-1 for 16 stations (All Belgian background PM10 stations)

Training period was 2000 - 2011 and testing 2012, because most of the PM10 stations started operating after 2000.

In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def smape(a, b):
    """
    Calculates sMAPE
    :param a: actual values
    :param b: predicted values
    :return: sMAPE
    """
    a = np.reshape(a, (-1,))
    b = np.reshape(b, (-1,))
    return 100.0 * np.mean(2.0 * np.abs(a - b) / (np.abs(a) + np.abs(b))).item()
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)    
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 # In %
def index_of_agreement(validation, prediction):
    return 1 - (np.sum((validation - prediction) ** 2)) / (np.sum((np.abs(prediction - 
      np.mean(validation)) + np.abs(validation - np.mean(validation))) ** 2))

In [2]:
from __future__ import print_function
import os
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot

# Make plots inline
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

modelLabel = 'rand'

dataManipulation = {
    "detrend": False,
    "scale": None,
#     "scale": 'standardize',
    # "scale": 'normalize',
    "swapEvery": 50000,  # Do swap island agent every iterations
    "master": 0,
    "folds": 10,
    "iterations": 800,
    "agents": 20
}

dataManipulation["scale"] = 'standardize'
year = 2012
lag = 1
    
print('Loading data...')
print('Using scale: {}'.format(dataManipulation["scale"]))

if dataManipulation["scale"] == 'standardize':
    r = np.genfromtxt("data/PM10_BETN_calendar_1995To2019/PM10_BETN_ts_standardized.csv", delimiter=',')
elif dataManipulation["scale"] == 'normalize':
    r = np.genfromtxt("data/PM10_BETN_calendar_1995To2019/PM10_BETN_ts_normalized.csv", delimiter=',')
else:
    r = np.genfromtxt("data/PM10_BETN/PM10_BETN_ts.csv", delimiter=',')
r = np.delete(r, [0], axis=1)  # Remove dates

# TODO: greatly decrease r length for testing: 2000-2012 training, 2013 for testing
row2000_01_01 = 5481 - 1
r = r[row2000_01_01:-1, :]

print('r shape:', r.shape)
print("\nStart Array r:\n {}".format(r[0, 0]))
maxLen = r.shape[1] - 1
print('Variables: {}'.format(maxLen))
print('TimeSteps: {}'.format(r.shape[0]))

mimoOutputs = 16
x_data = r[:, mimoOutputs:maxLen + 1]
y_data = r[:, 0:mimoOutputs]

print('x_data shape:', x_data.shape)
print("y_data shape:", y_data.shape)

y_data = np.array(y_data)
x_data_3d = x_data.reshape(x_data.shape[0], 1, x_data.shape[1])  # reshape input to 3D[samples, timesteps, features]

Loading data...
Using scale: standardize
r shape: (3285, 91)

Start Array r:
 -0.2514263162518368
Variables: 90
TimeSteps: 3285
x_data shape: (3285, 75)
y_data shape: (3285, 16)


In [3]:
from __future__ import print_function
import numpy as np
from keras.models import model_from_json
import pandas as pd

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Use CPU only for inference

json_file = open('runs/18CellularAutomata3DGrid3x3x3_5AgentsPM10_1994-2018_16_stations_calendar/local/foundModels/bestModelArchitecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("runs/18CellularAutomata3DGrid3x3x3_5AgentsPM10_1994-2018_16_stations_calendar/local/foundModels/bestModelWeights.h5")

Using TensorFlow backend.
W0725 06:10:52.438818  2828 deprecation_wrapper.py:119] From C:\Users\temp3rr0r\Anaconda3\envs\tensorflowLast\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



TypeError: ('Keyword argument not understood:', 'time_major')

In [ ]:
x_data, x_data_holdout = x_data_3d[:-365], x_data_3d[-365:]
y_data, y_data_holdout = y_data[:-365], y_data[-365:]

Note: The MAPE metric naturally goes to infinity, because there are valid sensor zero values ( see: https://en.wikipedia.org/wiki/Mean_absolute_percentage_error#Issues ) in the PM10 dataset. SMAPE is adviced for comparisons.

In [ ]:
holdout_prediction = loaded_model.predict(x_data_holdout)

sensor_mean = pd.read_pickle("data/PM10_BETN/PM10_BETN_ts_mean.pkl")
sensor_std = pd.read_pickle("data/PM10_BETN/PM10_BETN_ts_std.pkl")
sensor_mean = np.array(sensor_mean)
sensor_std = np.array(sensor_std)

predictionAll = (holdout_prediction * sensor_std[0:y_data.shape[1]]) + sensor_mean[0:y_data.shape[1]]
y_validationAll = (y_data_holdout * sensor_std[0:y_data.shape[1]]) + sensor_mean[0:y_data.shape[1]]
   
prediction = predictionAll
y_validation = y_validationAll

MSE = mean_squared_error(y_validation, prediction)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_validation, prediction)
MAPE = mean_absolute_percentage_error(y_validation, prediction)
SMAPE = ( smape(y_validation[:,0], prediction[:,0]) + smape(y_validation[:,1], prediction[:,1]) + smape(y_validation[:,2], prediction[:,2]) + smape(y_validation[:,3], prediction[:,3])) / 4.0
R2_score = r2_score(y_validation, prediction)
IOA = index_of_agreement(y_validation, prediction)

fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(20, 12), sharex=True, sharey=True)
ax.set_ylabel('Sensor value')
ax.set_xlabel('Day')
ax.set_title('PM10 Time-series (All {} Stations) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(mimoOutputs, year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
pyplot.plot(y_validation, color='blue')
ax.legend(['expected x {}'.format(mimoOutputs)])    

fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(20, 12), sharex=True, sharey=True)
ax.set_ylabel('Sensor value')
ax.set_xlabel('Day')
ax.set_title('PM10 Time-series (All {} Stations) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(mimoOutputs, year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
pyplot.plot(prediction, color='green')
ax.legend(['predicted x {}'.format(mimoOutputs)])    
  
fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(20, 12), sharex=True, sharey=True)
ax.set_ylabel('Sensor value')
ax.set_xlabel('Day')
ax.set_title('PM10 Time-series (All {} Stations) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(mimoOutputs, year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
ax.plot(y_validation[:,0], color='blue');
ax.plot(prediction[:,0], color='red');
ax.legend(['expected x {}'.format(mimoOutputs),'predicted x {}'.format(mimoOutputs)])
ax.plot(y_validation[:,1:None], color='blue');
ax.plot(prediction[:,1:None], color='red');
pyplot.savefig("PM10all16.svg")

In [ ]:
stations = ["BETN043", "BETN045", "BETN052", "BETN054", "BETN060", "BETN063", "BETN066", "BETN067", "BETN070", "BETN073", "BETN085", "BETN093", "BETN100", "BETN113", "BETN121", "BETN132"]
fig, ax = pyplot.subplots(nrows=len(stations), ncols=1, figsize=(20, 120), sharex=True, sharey=True)

for i in range(len(stations)):
    prediction = predictionAll[:,i]
    y_validation = y_validationAll[:,i]

    MSE = mean_squared_error(y_validation, prediction)
    RMSE = np.sqrt(MSE)
    MAE = mean_absolute_error(y_validation, prediction)
    MAPE = mean_absolute_percentage_error(y_validation, prediction)
    SMAPE = smape(y_validation, prediction)
    R2_score = r2_score(y_validation, prediction)
    IOA = index_of_agreement(y_validation, prediction)
    ax[i].set_ylabel('Sensor value')
    ax[i].set_title('PM10 Time-series (Station: {}) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(stations[i], year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
    ax[i].plot(y_validationAll[:,i])
    ax[i].plot(predictionAll[:,i])    
    ax[i].legend(['expected', 'predicted'])    
ax[len(stations)-1].set_xlabel('Day');

In [ ]:
stations = ["BETN043", "BETN045", "BETN052", "BETN054", "BETN060", "BETN063", "BETN066", "BETN067", "BETN070", "BETN073", "BETN085", "BETN093", "BETN100", "BETN113", "BETN121", "BETN132"]
fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(20, 12), sharex=True, sharey=True)
i = 0
prediction = predictionAll[:,i]
y_validation = y_validationAll[:,i]

MSE = mean_squared_error(y_validation, prediction)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_validation, prediction)
MAPE = mean_absolute_percentage_error(y_validation, prediction)
SMAPE = smape(y_validation, prediction)
R2_score = r2_score(y_validation, prediction)
IOA = index_of_agreement(y_validation, prediction)
ax.set_ylabel('Sensor value')
ax.set_title('PM10 Time-series (Station: {}) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(stations[i], year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
ax.plot(y_validationAll[:,i])
ax.plot(predictionAll[:,i])    
ax.legend(['expected', 'predicted'])
pyplot.savefig("pm10Brussels.svg")

In [ ]:
stations = ["BETN043", "BETN045", "BETN052", "BETN054", "BETN060", "BETN063", "BETN066", "BETN067", "BETN070", "BETN073", "BETN085", "BETN093", "BETN100", "BETN113", "BETN121", "BETN132"]
fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(20, 12), sharex=True, sharey=True)
i = 10
prediction = predictionAll[:,i]
y_validation = y_validationAll[:,i]

MSE = mean_squared_error(y_validation, prediction)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_validation, prediction)
MAPE = mean_absolute_percentage_error(y_validation, prediction)
SMAPE = smape(y_validation, prediction)
R2_score = r2_score(y_validation, prediction)
IOA = index_of_agreement(y_validation, prediction)
ax.set_ylabel('Sensor value')
ax.set_title('PM10 Time-series (Station: {}) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(stations[i], year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
ax.plot(y_validationAll[:,i])
ax.plot(predictionAll[:,i])    
ax.legend(['expected', 'predicted'])
pyplot.savefig("pm10Best.svg")

### Multi-step prediction:

Up to 7 days ahead, testing for 1 year:

In [ ]:
num_timeseries = 16
total_days = 365  # 365
max_days_ahead = 7  # 7
days = total_days - max_days_ahead

MSE_ahead = []
RMSE_ahead = []
MAE_ahead = []
smape_ahead = []
ioa_ahead = []
R2_score_ahead = []
y_ahead = []
y_hat_ahead = []

sensor_mean = pd.read_pickle("data/PM10_BETN/PM10_BETN_ts_mean.pkl")
sensor_std = pd.read_pickle("data/PM10_BETN/PM10_BETN_ts_std.pkl")
sensor_mean = np.array(sensor_mean)
sensor_std = np.array(sensor_std)

for days_ahead in range(1, max_days_ahead + 1):

    MSE_array = []
    RMSE_array = []
    MAE_array = []
    smape_array = []
    ioa_array = []
    R2_score_array = []
    y_array = []
    y_hat_array = []

    for day in range(0, days):
        x = []
        y = []
        y_hat = []
        x = np.array([x_data_holdout[day]])
        y = np.array(y_data_holdout[day + days_ahead - 1])
        y_hat = np.array(loaded_model.predict(x))

        for day_ahead in range(1, days_ahead):      
            x_next = np.array([x_data_holdout[day + day_ahead]])
            x_next[0, 0, -16:] = x[0, 0, :16]  # Current TS (first 16 columns) -> Previous TS (last 16 columns)
            x_next[0, 0, :16] = y_hat  # Predicted TS -> Current TS (first 16 columns)
            y_hat = np.array(loaded_model.predict(x_next))
        y_array.append(y)
        y_hat_array.append(y_hat[0])

    predictionAll = (y_hat_array * sensor_std[0:y_data.shape[1]]) + sensor_mean[0:y_data.shape[1]]
    y_validationAll = (y_array * sensor_std[0:y_data.shape[1]]) + sensor_mean[0:y_data.shape[1]]

    MSE = mean_squared_error(y_validationAll, predictionAll)
    RMSE = np.sqrt(MSE)
    MAE = mean_absolute_error(y_validationAll, predictionAll)
    smape_value = smape(y_validationAll, predictionAll)
    ioa = index_of_agreement(y_validationAll, predictionAll)
    R2_score = r2_score(y_validationAll, predictionAll)
    
    MSE_ahead.append(MSE)
    RMSE_ahead.append(RMSE)
    MAE_ahead.append(MAE)
    smape_ahead.append(smape_value)
    ioa_ahead.append(ioa)
    R2_score_ahead.append(R2_score)
    
    y_ahead.append(y_validationAll)
    y_hat_ahead.append(predictionAll)

In [ ]:
print("MSE_ahead", MSE_ahead)

In [ ]:
import matplotlib.pyplot as plt
red_square = dict(markerfacecolor='g', marker='o')
figure_size = (8, 6)

metric_strings = ["MSE", "RMSE", "MAE", "SMAPE", "IOA", "R2"]
metric_arrays = [MSE_ahead, RMSE_ahead, MAE_ahead, smape_ahead, ioa_ahead, R2_score_ahead]

for i in range(len(metric_strings)):

    fig, ax = plt.subplots(1, 1, figsize=figure_size)
    ax.set_xlabel("Value")
    ax.set_ylabel("Days Ahead")
    ax.xaxis.grid(True)    
    ax.set_title("Multi-day ahead prediction metric: {} (PM10: 1-day lag model, 16 stations, 2012 test)".format(metric_strings[i]))
    ax.barh(range(1, len(metric_arrays[i]) + 1), metric_arrays[i]);
    plt.yticks(range(1, len(metric_arrays[i]) + 1), range(1, len(metric_arrays[i]) + 1));

Plots:

In [ ]:
for i in range(len(MSE_ahead)):
    
    prediction = y_hat_ahead[i]
    y_validation = y_ahead[i]
    
    MSE = mean_squared_error(y_validation, prediction)
    RMSE = np.sqrt(MSE)
    MAE = mean_absolute_error(y_validation, prediction)
    MAPE = mean_absolute_percentage_error(y_validation, prediction)
    SMAPE = ( smape(y_validation[:,0], prediction[:,0]) + smape(y_validation[:,1], prediction[:,1]) + smape(y_validation[:,2], prediction[:,2]) + smape(y_validation[:,3], prediction[:,3])) / 4.0
    R2_score = r2_score(y_validation, prediction)
    IOA = index_of_agreement(y_validation, prediction)
    
    fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(20, 12), sharex=True, sharey=True)
    ax.set_ylabel('Sensor value')
    ax.set_xlabel('Day')
    ax.set_title('Days ahead {}: PM10 Time-series (All {} Stations) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(i + 1, mimoOutputs, year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
    pyplot.plot(y_validation, color='blue')
    ax.legend(['expected x {}'.format(mimoOutputs)])    

    fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(20, 12), sharex=True, sharey=True)
    ax.set_ylabel('Sensor value')
    ax.set_xlabel('Day')
    ax.set_title('Days ahead {}: PM10 Time-series (All {} Stations) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(i + 1, mimoOutputs, year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
    pyplot.plot(prediction, color='green')
    ax.legend(['predicted x {}'.format(mimoOutputs)])    

    fig, ax = pyplot.subplots(nrows=1, ncols=1, figsize=(20, 12), sharex=True, sharey=True)
    ax.set_ylabel('Sensor value')
    ax.set_xlabel('Day')
    ax.set_title('Days ahead {}: PM10 Time-series (All {} Stations) {}: Island LSTM MIMO NARX-{} prediction (MSE: {}, RMSE: {}, MAE: {}, MAPE: {}%, SMAPE: {}%, R2: {}, IOA: {}%)'.format(i + 1, mimoOutputs, year, lag, np.round(MSE,2), np.round(RMSE,2), np.round(MAE,2), np.round(MAPE,2), np.round(SMAPE,2), np.round(R2_score,2), np.round(IOA * 100,2)))
    ax.plot(y_validation[:,0], color='blue');
    ax.plot(prediction[:,0], color='red');
    ax.legend(['expected x {}'.format(mimoOutputs),'predicted x {}'.format(mimoOutputs)])
    ax.plot(y_validation[:,1:None], color='blue');
    ax.plot(prediction[:,1:None], color='red');
    pyplot.savefig("PM10all16.svg")

## Summary (PM10)

PM10 mean metrics for all the 16 background stations:

|Metric| Islands LSTM NARX-1 (All BE background 16 stations) |
|------|------|
|   **MSE**  | 116.01 |
|   **MAE**  | 7.62 |
|   **MAPE** | N/A |
|   **SMAPE**  | 30.27% |
|   **IoA**  | 83.68% |

### References:
[1] J. Orellana Alvear. Application of LS-SVMs to ozone forecasting in Belgium, 2015.  
[2] Denis Dumoulin. Fixed-Size Multi-Output LSSVM for Nonlinear System Identification, 2017.  
[3] Christoph Bergmeir, Rob J Hyndman, Bonsoo Koo (2018) A note on the validity of cross-validation for evaluating autoregressive time series prediction. Computational Statistics and Data Analysis, 120, 70-83.  
[4] Neural Architecture Search with Reinforcement Learning, Barret Zoph, Quoc V. Le. International Conference on Learning Representations, 2017.  
[5] Using Evolutionary AutoML to Discover Neural Network Architectures - https://ai.googleblog.com/2018/03/using-evolutionary-automl-to-discover.html  
[6] Ian Goodfellow, Yoshua Bengio, and Aaron Courville. 2016. Deep Learning. The MIT Press.  
[7] Esteban Real, Sherry Moore, Andrew Selle, Saurabh Saxena, Yutaka Leon Suematsu, Jie Tan, Quoc Le: "Large-Scale Evolution of Image Classifiers", 2017  
[8] Lingxue Zhu: "Deep and Confident Prediction for Time Series at Uber", 2017, 2017 IEEE International Conference on Data Mining Workshops. DOI: 10.1109/ICDMW.2017.19  
[9] Makridakis S, Spiliotis E, Assimakopoulos V (2018) Statistical and Machine Learning forecasting methods: Concerns and ways forward. PLOS ONE 13(3): e0194889. https://doi.org/10.1371/journal.pone.0194889  
[10] Esteban Real, Alok Aggarwal, Yanping Huang: "Regularized Evolution for Image Classifier Architecture Search", 2018  
[11] Xin Yao, "Evolving artificial neural networks," in Proceedings of the IEEE, vol. 87, no. 9, pp. 1423-1447, Sept. 1999.
doi: 10.1109/5.784219  
[12] Risto Miikkulainen, Jason Liang, Elliot Meyerson, Aditya Rawal, Dan Fink, Olivier Francon, Bala Raju, Hormoz Shahrzad, Arshak Navruzyan, Nigel Duffy: "Evolving Deep Neural Networks", 2017  
[13] Kenneth O. Stanley and Risto Miikkulainen: "Evolving Neural Networks Through Augmenting Topologies",  Evolutionary Computation  
[14] Variable Length Genomes for Evolutionary Algorithms –C. -y. Lee — 2000 — In Proceedings of the Genetic and Evolutionary Computation Conference, 806. Las Vegas  
[15] Tim Kovacs: Genetics-Based Machine Learning. Handbook of Natural Computing 2012: 937-986  
[16] Kennedy, J.; Eberhart, R. (1995). "Particle Swarm Optimization". Proceedings of IEEE International Conference on Neural Networks. IV. pp. 1942–1948. doi:10.1109/ICNN.1995.488968.  
[17] Storn, R., & Price, K. (1997). Differential Evolution – A Simple and Efficient Heuristic for global Optimization over Continuous Spaces. Journal of Global Optimization, 11(4), 341–359. https://doi.org/10.1023/A:1008202821328  
[18] P. A. Vikhar, "Evolutionary algorithms: A critical review and its future prospects," 2016 International Conference on Global Trends in Signal Processing, Information Computing and Communication (ICGTSPICC), Jalgaon, 2016, pp. 261-265.
doi: 10.1109/ICGTSPICC.2016.7955308  
[19] James Bergstra and Yoshua Bengio. 2012. Random search for hyper-parameter optimization. J. Mach. Learn. Res. 13 (February 2012), 281-305.  
[20] Wales, David J.; Doye, Jonathan P. K. (1997-07-10). "Global Optimization by Basin-Hopping and the Lowest Energy Structures of Lennard-Jones Clusters Containing up to 110 Atoms". The Journal of Physical Chemistry A. 101 (28): 5111–5116. doi:10.1021/jp970984n.  
[21] Stanley, Kenneth O. (2017-07-13). "Neuroevolution: A different kind of deep learning". O'Reilly Media. Retrieved 2017-09-04.  
[22] McCulloch, Warren; Walter Pitts (1943). "A Logical Calculus of Ideas Immanent in Nervous Activity". Bulletin of Mathematical Biophysics. 5 (4): 115–133. doi:10.1007/BF02478259.  
[23] Claesen, Marc, and Bart De Moor. "Hyperparameter Search in Machine Learning." arXiv preprint arXiv:1502.02127 (2015)  
[24] W.N. Martin, J. Lienig, and J.P. Cohoon. Island (migration) models: evolutionary algorithms based on punctuated equilibria. Evolutionary Computation 2: Advanced Algorithms and Operators. Institute of Physics Publishing, Bristol, 2000, chapter 15, pages 101–124.  
[25] Pelikan M. () Probabilistic Model-Building Genetic Algorithms. In: Hierarchical Bayesian Optimization Algorithm. Studies in Fuzziness and Soft Computing, vol 170. Springer, Berlin, Heidelberg  
[26] Pedro Larrañaga; Jose A. Lozano (2002). Estimation of Distribution Algorithms a New Tool for Evolutionary Computation. Boston, MA: Springer US. ISBN 978-1-4615-1539-5.  
[27] Sepp Hochreiter; Jürgen Schmidhuber (1997). "Long short-term memory". Neural Computation. 9 (8): 1735–1780. doi:10.1162/neco.1997.9.8.1735.
[28] Dee DP, Uppala SM, Simmons AJ, Berrisford P, Poli P, Kobayashi S, Andrae U, Balmaseda MA,Balsamo G, Bauer P, Bechtold P, Beljaars ACM, van deBerg L, Bidlot J, Bormann N, Delsol C, Dragani R,Fuentes  M,  Geer  AJ,  Haimberger  L,  Healy  SB,  Hersbach  H,  H ́olm  EV,  Isaksen  L,  K ̊allberg  P,  K ̈ohler  M,Matricardi M, McNally AP, Monge-Sanz BM, Morcrette J-J, Park B-K, Peubey C, de Rosnay P, Tavolato C,Th ́epaut  J-N,  Vitart  F.  2011.  The  ERA-Interim  reanalysis:  configuration  and  performance  of  the  dataassimilation system.Q. J. R. Meteorol. Soc.137: 553 – 597. DOI:10.1002/qj.828  
[29] Armstrong, J. S. (1978). Long-range Forecasting: From Crystal Ball to Computer, Wiley.  
[30] Tofallis, C (2015) "A Better Measure of Relative Prediction Accuracy for Model Selection and Model Estimation", Journal of the Operational Research Society, 66(8),1352-1362.  
[31] Willmott, C. J., S. G. Ackleson, R. E. Davis, J. J. Feddema, K. M. Klink, D. R. Legates, J. O'Donnell, and C. M. Rowe (1985), Statistics for the evaluation and comparison of models, J. Geophys. Res., 90(C5), 8995–9005, doi:10.1029/JC090iC05p08995.  
[32] Makridakis, Spyros, (1993), Accuracy measures: theoretical and practical concerns, International Journal of Forecasting, 9, issue 4, p. 527-529.  
[33] Tomassini, M. (2005). Spatially Structured Evolutionary Algorithms. New York. Springer. https://doi.org/10.1007/3-540-29938-6  
[34] Bergstra, J., Bardenet, R., Bengio, Y., & Kégl, B. (2011). Algorithms for Hyper-Parameter Optimization. In Proceedings of Neural Information Processing Systems (NIPS), 2011. https://doi.org/2012arXiv1206.2944S  
[35] Srivastava, N., Hinton, G., Krizhevsky, A., & Salakhutdinov, R. (2014). Dropout: A Simple Way to Prevent Neural Networks from Overfitting. Journal of Machine Learning Research (Vol. 15). Retrieved from http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf  
[36] Ioffe, S., & Szegedy, C. (2015). Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift. Retrieved from http://arxiv.org/abs/1502.03167  
[37] Klambauer, G., Unterthiner, T., Mayr, A., & Hochreiter, S. (2017). Self-Normalizing Neural Networks. Retrieved from http://arxiv.org/abs/1706.02515  
[38] Duchi JDUCHI, J., & Singer, Y. (2011). Adaptive Subgradient Methods for Online Learning and Stochastic Optimization * Elad Hazan. Journal of Machine Learning Research (Vol. 12).  
[39] Zeiler, M. D. (2012). ADADELTA: An Adaptive Learning Rate Method.  
[40] Kingma, D. P., & Ba, J. (2014). Adam: A Method for Stochastic Optimization.  
[41] Reddi, S. J., Kale, S., & Kumar, S. (2018). On the Convergence of Adam and Beyond. In Proceedings of the International Conference on Learning Representations (ICLR).  
[42] Sutskever, I., Martens, J., Dahl, G., & Hinton, G. (2013). On the importance of initialization and momentum in deep learning.  